In [9]:
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import os, glob, tqdm
import gc
import numpy as np
from diffsims.utils.sim_utils import get_electron_wavelength

In [26]:
root = r'G:\My Drive\PhD\projects\external_measurements\ml_difsims'
folder = 'data/experimental'
file_extension = '*.hspy'

path = os.path.join(root, folder, file_extension)
paths = glob.glob(path)
paths = [p for p in paths if 'radial' not in p]
paths.sort()
paths

['G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20200209_163154_centre_rebin_correct_rb_fullscan.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\20220126_142402_rebin_nav_2.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_3_rebin.hspy',
 'G:\\My Drive\\PhD\\projects\\external_measurements\\ml_difsims\\data/experimental\\roi_4_rebin.hspy']

In [49]:
from pyxem.detectors import Medipix515x515Detector

# Simulation microscope values (for azimuthal integration)
beam_energy = 200.0  #keV
detector_pix_size = 55e-6  #m

for p in tqdm.tqdm(paths[1:2]):
    print(p)
    dp = hs.load(p, signal_type='electron_diffraction')

    calibration = dp.axes_manager.signal_axes[0].scale
    detector_size = dp.axes_manager.signal_axes[0].size
    radial_steps = int(np.ceil((int(detector_size / 2) - 1) / 2) * 2)

    # # Old way (pyxem way):
    #dp.set_diffraction_calibration(calibration)
    dp.unit = "k_A^-1"
    dp.set_experimental_parameters(beam_energy=beam_energy)
    dp.set_ai(center=([detector_size / 2, detector_size / 2]))

    # New way (pyFAI)
    # detector = Medipix515x515Detector()
    # wavelength = get_electron_wavelength(beam_energy) * 1e-10
    # camera_length = detector_pix_size / (wavelength * calibration * 1e10)
    # center = ([detector_size / 2, detector_size / 2])
    # unit = "k_A^-1"
    # dp.unit = unit
    # ai = AzimuthalIntegrator(dist=camera_length, detector=detector, wavelength=wavelength)
    # ai.setFit2D(directDist=camera_length * 1000, centerX=center[1], centerY=center[0])
    # dp.metadata.set_item("Signal.ai", ai)

    radial = dp.get_azimuthal_integral1d(npt=radial_steps)
    print(radial)
    name = "{}_radial.hspy".format(os.path.basename(p).split('.')[0])
    radial.save(os.path.join(os.path.dirname(p), name), overwrite=True)
    gc.collect()


  0%|          | 0/1 [00:00<?, ?it/s]

G:\My Drive\PhD\projects\external_measurements\ml_difsims\data/experimental\20220126_142402_rebin_nav_2.hspy


C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\misc\utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


<ElectronDiffraction1D, title: , dimensions: (127, 127|160)>


ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: ai : Detector Detector	 Spline= None	 PixelSize= 1.379e-04, 1.379e-04 m
Wavelength= 2.507934e-12m
SampleDetDist= 1.000000e+00m	PONI= 2.206982e-02, 2.206982e-02m	rot1=0.000000  rot2= 0.000000  rot3= 0.000000 rad
DirectBeamDist= 1000.000mm	Center: x=160.000, y=160.000 pix	Tilt=0.000 deg  tiltPlanRotation= 0.000 deg
Traceback (most recent call last):
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\hyperspy\io_plugins\_hierarchical.py", line 774, in dict2group
    group.attrs[key] = value
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\h5py\_hl\attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "C:\Users\jf631\miniconda3\envs\pyxem_new\lib\site-packages\h5py\_hl\attrs.py", line 180

In [42]:
radial.isig[0.11:1.30].plot()
dp.plot()

In [50]:
radial.isig[0.11:1.30].sum().plot()

In [51]:
radial

<ElectronDiffraction1D, title: , dimensions: (127, 127|160)>